In [131]:
import folium
import pandas as pd
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
import re
import random

In [95]:
def get_district_links():
    url = "https://domokucha.info/moscow_metro_district.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    containers = soup.find_all('div', class_='container')

    district_links = []
    for container in containers:
        section = container.find('section')
        if section:
          links = section.find_all('a', href=True)
          for link in links:
              href = link['href']
              district_links.append(url.rsplit('/', 1)[0] + '/' + href)

    return district_links

In [96]:
district_links = get_district_links()
district_links

['https://domokucha.info/./moscow_metro_01_VAO.html',
 'https://domokucha.info/./moscow_metro_02_ZAO.html',
 'https://domokucha.info/./moscow_metro_03_SAO.html',
 'https://domokucha.info/./moscow_metro_04_SZAO.html',
 'https://domokucha.info/./moscow_metro_05_SVAO.html',
 'https://domokucha.info/./moscow_metro_06_CAO.html',
 'https://domokucha.info/./moscow_metro_07_UVAO.html',
 'https://domokucha.info/./moscow_metro_08_UZAO.html',
 'https://domokucha.info/./moscow_metro_09_UAO.html',
 'https://domokucha.info/./moscow_metro_10_Zelenograd.html',
 'https://domokucha.info/./moscow_metro_11_New-moscow.html',
 'https://domokucha.info/./moscow_metro_12_Troicky.html']

In [168]:
def parse_district_page(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    main = soup.find('main')
    stations = main.find_all('section')

    station_names = []
    for station in stations:
        station_name_elements = station.find_all('span', class_='features-title')

        for station_name_element in station_name_elements:
            station_name = station_name_element.text.strip()
            cleaned_name = re.sub(r'^\d+[\.\s]', '', station_name).strip()
            station_names.append(cleaned_name)

    return station_names

In [169]:
districts = {}
for district_link in district_links:
    print(f"Парсим округ: {district_link}")
    district_name = district_link.split("/")[-1].replace("moscow_metro_", "").replace(".html", "")
    districts[district_name] = parse_district_page(district_link),

Парсим округ: https://domokucha.info/./moscow_metro_01_VAO.html
Парсим округ: https://domokucha.info/./moscow_metro_02_ZAO.html
Парсим округ: https://domokucha.info/./moscow_metro_03_SAO.html
Парсим округ: https://domokucha.info/./moscow_metro_04_SZAO.html
Парсим округ: https://domokucha.info/./moscow_metro_05_SVAO.html
Парсим округ: https://domokucha.info/./moscow_metro_06_CAO.html
Парсим округ: https://domokucha.info/./moscow_metro_07_UVAO.html
Парсим округ: https://domokucha.info/./moscow_metro_08_UZAO.html
Парсим округ: https://domokucha.info/./moscow_metro_09_UAO.html
Парсим округ: https://domokucha.info/./moscow_metro_10_Zelenograd.html
Парсим округ: https://domokucha.info/./moscow_metro_11_New-moscow.html
Парсим округ: https://domokucha.info/./moscow_metro_12_Troicky.html


In [170]:
districts

{'01_VAO': (['Щёлковская',
   'Первомайская,',
   'Измайловская,',
   'Партизанская',
   'Семеновская',
   'Электрозаводская',
   'Новокосино',
   'Новогиреево',
   'Перово',
   'Шоссе Энтузиастов',
   'Бульвар Рокоссовского',
   'Черкизовская',
   'Преображенская Площадь',
   'Сокольники',
   'Выхино'],),
 '02_ZAO': (['Крылатское',
   'Молодежная',
   'Кунцевская',
   'Славянский бульвар',
   'Парк Победы',
   'Киевская',
   'Киевская',
   'Воробьёвы горы',
   'Университет',
   'Проспект Вернадского',
   'Юго-Западная',
   'Тропарёво',
   'Кунцевская',
   'Пионерская,',
   'Филевский Парк',
   'Багратионовская',
   'Фили',
   'Кутузовская',
   'Студенческая',
   'Киевская',
   'Кутузовcкая',
   'Площадь Гагарина'],),
 '03_SAO': (['Савёловская',
   'Ховрино',
   'Беломорская',
   'Речной вокзал',
   'Водный стадион',
   'Войковская',
   'Сокол',
   'Аэропорт',
   'Динамо',
   'Полежаевская',
   'Беговая',
   'Петровско-Разумовская',
   'Тимирязевская',
   'Савёловская',
   'Тимирязевск

In [135]:
metro_data = pd.read_csv('metro.tsv', sep='\t')
metro_data

,Название,Широта,Долгота
0,Аэропорт,55.799810,37.534120
1,Академическая,55.688080,37.575010
2,Алексеевская,55.807370,37.638440
3,Александровский сад,55.752190,37.608360
4,Алтуфьево,55.895040,37.586050
...,...,...,...
160,Воробьёвы горы,55.710454,37.558601
161,Выхино,55.715000,37.818020
162,Ясенево,55.605350,37.534940
163,Юго-западная,55.664640,37.484210


In [178]:
colors = [
    "red", "green", "blue", "orange", "purple", "darkblue", "darkgreen", "darkred",
    "lightred", "beige", "darkpurple", "white", "pink", "cadetblue", "lightblue",
    "lightgreen", "gray", "black", "lightgray"
]

def get_unique_color(assigned_colors):
    available_colors = [color for color in colors if color not in assigned_colors]
    if available_colors:
        color = random.choice(available_colors)
        assigned_colors.add(color)
        return color

In [179]:
map_moscow = folium.Map(location=[55.7558, 37.6173], zoom_start=11)
assigned_colors = set()

for district, (stations,) in districts.items():
    district_color = get_unique_color(assigned_colors)
    for station in stations:
        station_data = metro_data[metro_data['Название'] == station]

        if not station_data.empty:
            lat = station_data.iloc[0]['Широта']
            lon = station_data.iloc[0]['Долгота']
            print(district_color)
            folium.Marker(
                location=[lat, lon],
                popup=f"{station}<br>Координаты: {lat}, {lon}, {district_color}",
                icon=folium.Icon(color=district_color)
            ).add_to(map_moscow)
        # else:
            # print(station)

darkgreen
darkgreen
darkgreen
darkgreen
darkgreen
darkgreen
lightgray
lightgray
lightgray
lightgray
lightgray
lightgray
lightgray
lightgray
lightgray
lightgray
lightgray
lightgray
lightgray
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
lightgreen
beige
beige
beige
beige
beige
green
green
green
green
green
green
green
green
green
green
green
green
green
green
green
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
darkpurple
pink
pink
pink
pink
pink
pink
pink
pink
pink
pink
red
red
red
red
red
red
red
red
red
lightred
lightred
lightred
lightred
lightred
lightred
lightred
lightred
ligh

In [180]:
map_moscow